- 플레이리스트 데이터 셋을 가지고 각 노래의 임베딩을 구함

- 플레이리스트가 서로 유사한 장르, 분위기, 태그 등을 가진 노래로 구성됨

- Word2Vec Model을 이용하여 노래의 임베딩을 구함

- 구해진 노래의 임베딩은 단순히 장르의 특징만을 가지진 않고 장르, 분위기, 태그 등의 특징을 가짐

In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import warnings

warnings.filterwarnings(action='ignore')

data_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Data/'
model_dir = '/content/drive/MyDrive/제 13회 투빅스 컨퍼런스 음악추천/Model/'

# 데이터 확인

In [ ]:
# 노래 데이터 불러오기
song_meta_df = pd.read_json(data_dir + 'song_meta_data_v3.json')
song_meta_df = song_meta_df.sort_values('id')
song_meta_df = song_meta_df.reset_index(drop = True)
song_meta_df['song_embedding_idx'] = song_meta_df.index

In [ ]:
# 플레이리스트 데이터 불러오기
playlist_df = pd.read_json(data_dir + 'playlist_data_v1.json')

In [ ]:
# 노래 데이터에만 존재하는 플레이리스트 데이터 만들기
song_li = set(song_meta_df['id'].astype(str).tolist())
playlist_song_li = playlist_df['songs'].values
train_playlist_song_li = []

for playlist in tqdm(playlist_song_li):
    playlist = set(playlist)
    new_playlist = list(song_li & playlist)
    train_playlist_song_li.append(list(map(str, new_playlist)))

# 학습

In [ ]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from gensim.models.callbacks import CallbackAny2Vec
import datetime

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 1
        self.loss_to_be_subed = 0
        self.loss_now = 987654321

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        loss_now = loss - self.loss_to_be_subed
        print(f'Loss after epoch {self.epoch}: {loss_now}, Total_loss : {loss}')

        self.loss_to_be_subed = loss
        if loss_now < self.loss_now:
            self.loss_now = loss_now
            model.wv.save_word2vec_format(model_dir + 'Word2Vec_Music_Model')
            print('Model 저장')
        self.epoch += 1

start = datetime.datetime.now()

model = Word2Vec(
                sentences = train_playlist_song_li,
                seed = 22,
                iter = 30,
                min_count = 1,
                size = 128,
                sg = 1,
                negative = 5,
                window = 10,
                compute_loss=True, 
                callbacks=[callback()],
                 )

print("Time passed: " + str(datetime.datetime.now() - start))

# Embedding 저장

In [ ]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

model = KeyedVectors.load_word2vec_format(model_dir + 'Word2Vec_Music_Model')

print('학습된 전체 노래의 개수: ', len(model.wv.vocab))

In [ ]:
id_li = song_meta_df['id'].astype(str).tolist()
word2vec_embedding = []
for id in id_li:
    word2vec_embedding.append(model[id].reshape(1, -1))

In [ ]:
word2vec_embedding = np.concatenate(word2vec_embedding)
np.save(data_dir + 'word2vec_embedding.npy', word2vec_embedding)

In [ ]:
word2vec_embedding.shape